In [1]:
from utils.apps import SimpleApp
from langchain_openai import ChatOpenAI
from utils.db import get_database_schema_execute_all
import os
from keys import OPENAI_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_KEY  

In [14]:
# Change path to csv files and prompt
path_to_csv_files = 'testDBs/test1/db/'
path_to_csv_schema_file = 'testDBs/test1/csv_schema.xlsx'
path_to_groud_truth_eventlog = 'testDBs/test1/ground-truth-eventlog.csv'
db_output_dir = 'example.db'
llm_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) #gpt-4, gpt-3.5-turbo
db_schema = get_database_schema_execute_all(path_to_csv_files = path_to_csv_files,path_to_csv_schema_file=path_to_csv_schema_file, db_output_dir= db_output_dir)
prompt=f"""Consider the following db schema:
            {db_schema}
            Each record in each table represents at least one event.
            If a table contains multiple timestamp attributes, then each of these attributes is related to an event with the activity_id equal to the attribute name.
            
            Use the id of the order as the case notion.
            
            When a table does not have an attribute that contains the selected case notion, combine the necessary tables to obtain this case notion.
            
            Write one sql statement that returns an event log from the selected tables with the following columns: case_id,activity_id,timestamp. 
            In activity_id leave out "_created".
            Use quotes for identifiers. Provide only the query."""


CREATE TABLE "order" ("id" PRIMARY KEY, "creation_date" DATETIME)
CREATE TABLE "payment" ("id" PRIMARY KEY, "invoice_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("invoice_id") REFERENCES "invoice"("id"))
CREATE TABLE "shipment" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("order_id") REFERENCES "order"("id"))
CREATE TABLE "invoice" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("order_id") REFERENCES "order"("id"))
Data inserted into table order
Data inserted into table invoice
Data inserted into table payment
Data inserted into table shipment
Database created: example.db


In [15]:
print(prompt)

Consider the following db schema:
            TABLE invoice:
CREATE TABLE "invoice" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("order_id") REFERENCES "order"("id"));

TABLE order:
CREATE TABLE "order" ("id" PRIMARY KEY, "creation_date" DATETIME);

TABLE payment:
CREATE TABLE "payment" ("id" PRIMARY KEY, "invoice_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("invoice_id") REFERENCES "invoice"("id"));

TABLE shipment:
CREATE TABLE "shipment" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("order_id") REFERENCES "order"("id"));

            Each record in each table represents at least one event.
            If a table contains multiple timestamp attributes, then each of these attributes is related to an event with the activity_id equal to the attribute name.
            
            Use the id of the order as the case notion.
            
            When a table does not have an attribute that contains the selected case notio

In [16]:
AgentState = {"messages": [prompt]}
a = SimpleApp(path_to_db = db_output_dir, path_to_groud_truth_eventlog=path_to_groud_truth_eventlog, llm_model = llm_model)
result = a.invoke(AgentState)

Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers


In [17]:
result

{'messages': ['Consider the following db schema:\n            TABLE invoice:\nCREATE TABLE "invoice" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("order_id") REFERENCES "order"("id"));\n\nTABLE order:\nCREATE TABLE "order" ("id" PRIMARY KEY, "creation_date" DATETIME);\n\nTABLE payment:\nCREATE TABLE "payment" ("id" PRIMARY KEY, "invoice_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("invoice_id") REFERENCES "invoice"("id"));\n\nTABLE shipment:\nCREATE TABLE "shipment" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("order_id") REFERENCES "order"("id"));\n\n            Each record in each table represents at least one event.\n            If a table contains multiple timestamp attributes, then each of these attributes is related to an event with the activity_id equal to the attribute name.\n            \n            Use the id of the order as the case notion.\n            \n            When a table does not have an attribute that

In [18]:
result['sqlexecuter']

,case_id,activity_id,timestamp
0,o1,order,2017-08-11 10:33:37
1,o2,order,2017-08-13 16:18:15
2,o1,invoice,2017-08-15 09:13:27
3,o3,order,2017-08-15 22:02:53
4,o4,order,2017-08-17 02:18:12
5,o2,invoice,2017-08-17 17:38:36
6,o3,invoice,2017-08-19 23:03:45
7,o4,invoice,2017-08-21 14:25:04
8,o1,payment,2017-08-21 16:26:13
9,o3,payment,2017-08-23 18:42:09


In [19]:
result['result']

{'prec': 0.0, 'rec': 0.0, 'f1': 0}